In [7]:
import pandas as pd
import numpy as np

In [8]:
initial_df = pd.read_csv('Complete_BF_CPP_XY_Data.csv')
complete_df = initial_df.drop_duplicates(subset=[initial_df.columns[1], initial_df.columns[2]], keep='first').reset_index(drop=True)
print(len(complete_df))

15792


In [9]:
# Interpolator takes a train_df, and uses 4 inputs as longitude, latitude, county_fips and state_fips. It outputs 22 interpolated outputs.
# Define the function
def get_location_data(df, longitude, latitude, county_fips, state_fips):
    # Filter for rows with the specified state FIPS
    matched_rows = df[df.iloc[:, 4] == state_fips]
    
    # Initialize outputs with zeros for indices not yet computed
    outputs = [0] * 22

    if not matched_rows.empty:
        # Get the first match for specified columns
        matched_row = matched_rows.iloc[0]
        outputs[0] = matched_row.iloc[5]   # 5th column in df
        outputs[1] = matched_row.iloc[6]   # 6th column in df
        outputs[2] = matched_row.iloc[7]   # 7th column in df
        outputs[3] = matched_row.iloc[8]   # 8th column in df
        outputs[5] = matched_row.iloc[10]  # 10th column in df
        outputs[6] = matched_row.iloc[11]  # 11th column in df

    # Filter for rows with the same county FIPS
    county_rows = df[df.iloc[:, 3] == county_fips]  # Adjusted column for county FIPS

    # Compute the Euclidean distance for each row in county_rows
    if not county_rows.empty:
        distances = np.sqrt((county_rows.iloc[:, 1] - longitude) ** 2 + (county_rows.iloc[:, 2] - latitude) ** 2)
        
        if len(county_rows) == 1:
            # Single entry in the same county
            single_row = county_rows.iloc[0]
            outputs[4] = single_row.iloc[9]
            outputs[7] = single_row.iloc[12]
            outputs[8] = single_row.iloc[13]
            outputs[14] = single_row.iloc[19]
            outputs[16] = single_row.iloc[21]
            outputs[17] = single_row.iloc[22]
            outputs[18] = single_row.iloc[23]
            outputs[19] = single_row.iloc[24]
            outputs[20] = single_row.iloc[25]

            outputs[9] = single_row.iloc[14]
            outputs[10] = single_row.iloc[15]
            outputs[11] = single_row.iloc[16]
            outputs[12] = single_row.iloc[17]
            outputs[13] = single_row.iloc[18]
            outputs[15] = single_row.iloc[20]
            outputs[21] = single_row.iloc[26]
        
        else:
            # More than one entry in the same county, select the two closest entries
            closest_indices = distances.nsmallest(2).index
            loc1 = county_rows.loc[closest_indices[0]]
            loc2 = county_rows.loc[closest_indices[1]]
            
            # Calculate distances for interpolation weights
            dist1 = distances.loc[closest_indices[0]]
            dist2 = distances.loc[closest_indices[1]]
            weight1 = dist2 / (dist1 + dist2)
            weight2 = dist1 / (dist1 + dist2)
            
            # Perform linear interpolation for each specified output index
            outputs[4] = loc1.iloc[9] * weight1 + loc2.iloc[9] * weight2
            outputs[7] = loc1.iloc[12] * weight1 + loc2.iloc[12] * weight2
            outputs[8] = loc1.iloc[13] * weight1 + loc2.iloc[13] * weight2
            outputs[14] = loc1.iloc[19] * weight1 + loc2.iloc[19] * weight2
            outputs[16] = loc1.iloc[21] * weight1 + loc2.iloc[21] * weight2
            outputs[17] = loc1.iloc[22] * weight1 + loc2.iloc[22] * weight2
            outputs[18] = loc1.iloc[23] * weight1 + loc2.iloc[23] * weight2
            outputs[19] = loc1.iloc[24] * weight1 + loc2.iloc[24] * weight2
            outputs[20] = loc1.iloc[25] * weight1 + loc2.iloc[25] * weight2
    
            outputs[9] = loc1.iloc[14] * weight1 + loc2.iloc[14] * weight2
            outputs[10] = loc1.iloc[15] * weight1 + loc2.iloc[15] * weight2
            outputs[11] = loc1.iloc[16] * weight1 + loc2.iloc[16] * weight2
            outputs[12] = loc1.iloc[17] * weight1 + loc2.iloc[17] * weight2
            outputs[13] = loc1.iloc[18] * weight1 + loc2.iloc[18] * weight2
            outputs[15] = loc1.iloc[20] * weight1 + loc2.iloc[20] * weight2
            outputs[21] = loc1.iloc[26] * weight1 + loc2.iloc[26] * weight2

    else:
        # No entry with the same county FIPS, find the 2 closest locations across the whole dataset
        distances_all = np.sqrt((df.iloc[:, 1] - longitude) ** 2 + (df.iloc[:, 2] - latitude) ** 2)
        closest_indices_all = distances_all.nsmallest(2).index
        loc1 = df.loc[closest_indices_all[0]]
        loc2 = df.loc[closest_indices_all[1]]
        
        # Calculate distances for interpolation weights
        dist1 = distances_all[closest_indices_all[0]]
        dist2 = distances_all[closest_indices_all[1]]
        weight1 = dist2 / (dist1 + dist2)
        weight2 = dist1 / (dist1 + dist2)
        
        # Perform interpolation
        outputs[4] = loc1.iloc[9] * weight1 + loc2.iloc[9] * weight2
        outputs[7] = loc1.iloc[12] * weight1 + loc2.iloc[12] * weight2
        outputs[8] = loc1.iloc[13] * weight1 + loc2.iloc[13] * weight2
        outputs[14] = loc1.iloc[19] * weight1 + loc2.iloc[19] * weight2
        outputs[16] = loc1.iloc[21] * weight1 + loc2.iloc[21] * weight2
        outputs[17] = loc1.iloc[22] * weight1 + loc2.iloc[22] * weight2
        outputs[18] = loc1.iloc[23] * weight1 + loc2.iloc[23] * weight2
        outputs[19] = loc1.iloc[24] * weight1 + loc2.iloc[24] * weight2
        outputs[20] = loc1.iloc[25] * weight1 + loc2.iloc[25] * weight2

        outputs[9] = loc1.iloc[14] * weight1 + loc2.iloc[14] * weight2
        outputs[10] = loc1.iloc[15] * weight1 + loc2.iloc[15] * weight2
        outputs[11] = loc1.iloc[16] * weight1 + loc2.iloc[16] * weight2
        outputs[12] = loc1.iloc[17] * weight1 + loc2.iloc[17] * weight2
        outputs[13] = loc1.iloc[18] * weight1 + loc2.iloc[18] * weight2
        outputs[15] = loc1.iloc[20] * weight1 + loc2.iloc[20] * weight2
        outputs[21] = loc1.iloc[26] * weight1 + loc2.iloc[26] * weight2

    # Round specified indices before returning
    outputs[0] = int(round(outputs[0]))
    outputs[3] = int(round(outputs[3]))
    outputs[5] = int(round(outputs[5]))
    outputs[18] = int(round(outputs[18]))

    outputs[9] = int(round(outputs[9]))
    outputs[10] = int(round(outputs[10]))
    outputs[11] = int(round(outputs[11]))
    outputs[12] = int(round(outputs[12]))
    outputs[13] = int(round(outputs[13]))
    outputs[15] = int(round(outputs[15]))
    outputs[20] = int(round(outputs[20]))

    # Remove indices 9, 10, 11, 12, 13, 15, and 21
    #for index in sorted([21, 15, 13, 12, 11, 10, 9], reverse=True):
    #    del outputs[index]

    return outputs

In [10]:
# Example usage
# Replace with actual longitude, latitude, county_fips, and state_fips values as needed
result = get_location_data(complete_df, longitude=-92.366, latitude=42.537, county_fips=19013, state_fips=19) # index 10563 in excel data table 
print(result)

[0, 9.31923076923077, -5401.0, 0, 0.28898115180657524, 1, 37572.0, 0.0, 2.1763964909602795, 1, 1, 1, 0, 1, 0.4045521815239518, 1, 4.078753349496401, 57.58192338913222, 0, 0.8555120539666661, 2, 1.0]


In [11]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.model_selection import train_test_split

In [22]:
# Split the dataset into training (85%) and test (15%) sets
train_df, test_df = train_test_split(complete_df, test_size=0.15, random_state=42)

# Extract input columns (longitude, latitude, county_fips, state_fips)
train_inputs = train_df.iloc[:, [1, 2, 3, 4]].values
test_inputs = test_df.iloc[:, [1, 2, 3, 4]].values

# Run predictions with the function on test inputs, using train_df as the df for interpolation
predictions = np.array([get_location_data(train_df, *inputs) for inputs in test_inputs])

In [23]:
# Extract objective columns for the test set
# Select only the columns [0, 1, 2, 3, 4, 5, 6, 7, 8, 14, 16, 17, 18, 19, 20] for comparison
test_targets = test_df.iloc[:, [5, 6, 7, 8, 9, 10, 11, 12, 13, 19, 21, 22, 23, 24, 25]].values
binary_targets = test_df.iloc[:, [14, 15, 16, 17, 18, 20, 26]].values

L_predictions = predictions[:,[0,1,2,3,4,5,6,7,8,14,16,17,18,19,20]]
B_predictions = predictions[:,[9, 10, 11, 12, 13, 15, 21]]

In [24]:
# Calculate error metrics
mae = mean_absolute_error(test_targets, L_predictions)
mse = mean_squared_error(test_targets, L_predictions)
rmse = np.sqrt(mse)
r2 = r2_score(test_targets, L_predictions)

# Print the results
print("Mean Absolute Error (MAE):", mae)
print("Mean Squared Error (MSE):", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("R-squared (R2):", r2)

Mean Absolute Error (MAE): 0.5568929354730567
Mean Squared Error (MSE): 6.9679600240699155
Root Mean Squared Error (RMSE): 2.639689380224483
R-squared (R2): 0.8151878980557999


In [27]:
from sklearn.metrics import accuracy_score
binary_predictions = (B_predictions >= 0.5).astype(int)
# Loop through each column index and calculate accuracy
for i in range(binary_targets.shape[1]):
    accuracy = accuracy_score(binary_targets[:, i], binary_predictions[:, i])
    print(f"Accuracy for column {i + 1}: {accuracy:.4f}")

Accuracy for column 1: 0.9899
Accuracy for column 2: 0.9472
Accuracy for column 3: 0.9954
Accuracy for column 4: 0.9498
Accuracy for column 5: 0.8907
Accuracy for column 6: 0.9316
Accuracy for column 7: 0.9747
